# CAI Prompt

In [ ]:
import os
import openai
from dotenv import load_dotenv
import concurrent.futures
load_dotenv()

number_prompts = 5

: 

## Generate tasks

In [ ]:


openai.api_key = os.getenv("OPENAI_API_KEY")

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
  "role": "user",
     "content": f"Generate a list of {number_prompts} tasks for a customer service assistant involving reading and writing. Write each task on a separate line and number each task."}
  ]
)

tasks_string = completion.choices[0].message.content
print(tasks_string)

: 

In [ ]:
def getItems(str):
    items = []
    for line in str.splitlines():
        items.append(line[line.find(".") + 2:])
    return items

tasks = getItems(tasks_string)
print(len(tasks))

: 

## Generate naive prompts

> **Terminology** - here we're prompting an LLM to generate prompts, which is confusing. We'll use the word "prompt" to refer to the initial input, and "response" to refer to the output

In [ ]:
def get_llm_response(task, i):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": f"Here is a task for a customer service assistant:\n\n{task}\n\nWrite a prompt for a large language model (LLM), asking it to carry out this task."
            }
        ],
        max_tokens=1024,
        temperature=0.5,
        stream=False
    )
    str = response['choices'][0]['message']['content']
    print(f"Response {i} received")
    return str

with concurrent.futures.ThreadPoolExecutor() as executor:
    naive_responses = list(executor.map(lambda x: get_llm_response(x[0], x[1]), [(task, i) for i, task in enumerate(tasks)]))

tasksResponsesString = "\n\n---\n\n".join([f"TASK:\n{task}\n\nNAIVE PROMPT:\n{response}" for task, response in zip(tasks, naive_responses)])
print(tasksResponsesString)

: 

Most of these responses are slightly more detailed rewrites of the corresponding task. This is a good start, but ultimately not very useful and doesn't demonstrate most of the good practices in prompt design.